In [1]:
import mlflow
import os
import pandas as pd

from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature
from mlflow.store.artifact.artifact_repository_registry import get_artifact_repository

from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
os.getenv("MLFLOW_TRACKING_URI", "No env")

'http://mlflow-service:5000'

In [3]:
mlflow.get_registry_uri()

'http://mlflow-service:5000'

# Управление экспериментами

In [9]:
exp_id = mlflow.create_experiment(name="test")

In [10]:
exp_id

'958964435771505022'

In [1]:
# mlflow.delete_experiment(experiment_id=exp_id)

Не забудьте про rm -rf mlruns/trash/*

In [12]:
mlflow.search_experiments(
    filter_string="name = 'test'"
)

[<Experiment: artifact_location='s3://kirillbucket/mlflow/958964435771505022', creation_time=1732195333448, experiment_id='958964435771505022', last_update_time=1732195333448, lifecycle_stage='active', name='test', tags={}>]

In [13]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://kirillbucket/mlflow/958964435771505022', creation_time=1732195333448, experiment_id='958964435771505022', last_update_time=1732195333448, lifecycle_stage='active', name='test', tags={}>,
 <Experiment: artifact_location='s3://kirillbucket/mlflow/0', creation_time=1732185954947, experiment_id='0', last_update_time=1732185954947, lifecycle_stage='active', name='Default', tags={}>]

In [14]:
mlflow.set_experiment(experiment_name="test")

with mlflow.start_run():
    # Обучим модель.
    model.fit(X_train, y_train)

2024/11/21 13:26:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run burly-trout-19 at: http://mlflow-service:5000/#/experiments/958964435771505022/runs/3b4400e7dfb84cf7a35025c4e751f342.
2024/11/21 13:26:10 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/958964435771505022.


NameError: name 'model' is not defined

# Что можно отслеживать в рамках эксперимента

In [15]:
# Прочитаем данные.
housing = fetch_california_housing(as_frame=True)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(housing['data'], housing['target'])
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [17]:
X_test.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
2435,3.5987,19.0,6.167260,1.078292,911.0,3.241993,36.57,-119.59
890,6.6290,25.0,6.154545,0.872727,596.0,2.709091,37.54,-121.93
1310,3.3625,13.0,5.044395,1.056604,2188.0,2.428413,37.99,-121.84
5907,4.6864,35.0,6.450495,1.106436,1555.0,3.849010,34.29,-118.44
2089,2.1055,52.0,5.178495,1.070968,1361.0,2.926882,36.76,-119.79


## Первый запуск run

In [19]:
model = LinearRegression()

In [20]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run(experiment_id=exp_id) as run:
    # Обучим модель.
    model.fit(X_train, y_train)

2024/11/21 13:28:20 INFO mlflow.tracking._tracking_service.client: 🏃 View run righteous-whale-229 at: http://mlflow-service:5000/#/experiments/958964435771505022/runs/55d1a97b399d4f7db193485c0e09f17f.
2024/11/21 13:28:20 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/958964435771505022.


## Ручное логирование метрик

In [21]:
model = RandomForestRegressor()

In [22]:
params = {'n_estimators':50, 'min_samples_split':3}

In [23]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run(experiment_id=exp_id) as run:

    # Обучим модель.
    model.fit(X_train, y_train)

    # Сделаем предсказание.
    prediction = model.predict(X_val)

    mlflow.log_metrics({'r2':r2_score(y_val, prediction),
                        'mse':mean_squared_error(y_val, prediction)})

    mlflow.log_params(params)

2024/11/21 13:30:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run placid-yak-867 at: http://mlflow-service:5000/#/experiments/958964435771505022/runs/c9a950071976472eb952c5f003bd660e.
2024/11/21 13:30:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/958964435771505022.


## Автоматическая оценка модели

In [24]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run(experiment_id=exp_id) as run:
    # Обучим модель.
    model.fit(X_train, y_train)

    # Сделаем предсказание.
    prediction = model.predict(X_val)

    # Создадим валидационный датасет.
    eval_df = X_val.copy()
    eval_df["target"] = y_val
    eval_df["prediction"] = prediction

    # Сохраним результаты обучения с помощью MLFlow.
    mlflow.evaluate(
        data=eval_df,
        targets="target",
        predictions="prediction",
        model_type="regressor",
        evaluators=["default"],
    )

Matplotlib is building the font cache; this may take a moment.
2024/11/21 13:51:16 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/21 13:51:16 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/11/21 13:51:26 INFO mlflow.tracking._tracking_service.client: 🏃 View run smiling-fox-610 at: http://mlflow-service:5000/#/experiments/958964435771505022/runs/3df8075f35584a648e6ba23f1e9310c5.
2024/11/21 13:51:26 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/958964435771505022.


## Логирование модели в хранилище артефактов

In [160]:
model = LinearRegression()

In [25]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run(experiment_id=exp_id) as run:
    # Обучим модель.
    model.fit(X_train, y_train)

    # Сделаем предсказание.
    prediction = model.predict(X_val)

    # Создадим валидационный датасет.
    eval_df = X_val.copy()
    eval_df["target"] = y_val
    #eval_df["prediction"] = prediction

    # Сохраним результаты обучения с помощью MLFlow.
    signature = infer_signature(housing['data'], prediction)
    model_info = mlflow.sklearn.log_model(model, 'logreg', signature=signature)
    mlflow.evaluate(
        model=model_info.model_uri,
        data=eval_df,
        targets="target",
        #predictions="prediction",
        model_type="regressor",
        evaluators=["default"],
    )

2024/11/21 13:59:26 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/21 13:59:27 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/21 13:59:27 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/11/21 13:59:28 INFO mlflow.tracking._tracking_service.client: 🏃 View run secretive-worm-327 at: http://mlflow-service:5000/#/experiments/958964435771505022/runs/29fbb845cc0b48a8b6f4dd330ed068a7.
2024/11/21 13:59:28 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/958964435771505022.


In [26]:
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"

In [27]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run(experiment_id=exp_id) as run:
    # Обучим модель.
    model.fit(X_train, y_train)

    # Сделаем предсказание.
    prediction = model.predict(X_val)

    # Создадим валидационный датасет.
    eval_df = X_val.copy()
    eval_df["target"] = y_val
    #eval_df["prediction"] = prediction	Michael «Mike» Score
    # Сохраним результаты обучения с помощью MLFlow.
    signature = infer_signature(housing['data'], prediction)
    model_info = mlflow.sklearn.log_model(model, 'logreg', signature=signature)
    mlflow.evaluate(
        model=model_info.model_uri,
        data=eval_df,
        targets="target",
        #predictions="prediction",
        model_type="regressor",
        evaluators=["default"],
    )

2024/11/21 14:18:20 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/11/21 14:18:20 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing d

2024/11/21 14:19:22 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/21 14:19:23 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/21 14:19:23 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/11/21 14:19:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run placid-eel-731 at: http://mlflow-service:5000/#/experiments/958964435771505022/runs/1db8588fe7124df3a5717904aefbc45d.
2024/11/21 14:19:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/958964435771505022.
2024/11/21 14:19:23 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/21 14:19:23 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [164]:
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "false"

In [34]:
# Удалим эксперимент из MLFlow.
mlflow.delete_experiment(experiment_id=exp_id)

NameError: name 'experiment_id' is not defined

In [166]:
# Удалим эксперимент из S3.
repository = get_artifact_repository(f's3://lizvladi-mlflow-artifacts/mlflow/{exp_id}')
repository.delete_artifacts()

## Работа с дочерними ранами

In [37]:
# Создать новый эксперимент
exp_name = "parent_run_experiment"
experiment_id = mlflow.create_experiment(exp_name)
mlflow.set_experiment(exp_name)

<Experiment: artifact_location='s3://kirillbucket/mlflow/270645224134058005', creation_time=1732199336800, experiment_id='270645224134058005', last_update_time=1732199336800, lifecycle_stage='active', name='parent_run_experiment', tags={}>

In [38]:
models = dict(zip(["RandomForest", "LinearRegression", "HistGB"], 
                  [RandomForestRegressor(), LinearRegression(), HistGradientBoostingRegressor()]))

In [39]:
# Создадим parent run.
with mlflow.start_run(run_name="parent_run", experiment_id = experiment_id, description = "parent") as parent_run:
    for model_name in models.keys():
        # Запустим child run на каждую модель.
        with mlflow.start_run(run_name=model_name, experiment_id=experiment_id, nested=True) as child_run:
            model = models[model_name]
            
            # Обучим модель.
            model.fit(pd.DataFrame(X_train), y_train)
        
            # Сделаем предсказание.
            prediction = model.predict(X_val)
        
            # Создадим валидационный датасет.
            eval_df = X_val.copy()
            eval_df["target"] = y_val
        
            # Сохраним результаты обучения с помощью MLFlow.
            signature = infer_signature(X_test, prediction)
            model_info = mlflow.sklearn.log_model(model, "logreg", signature=signature)
            mlflow.evaluate(
                model=model_info.model_uri,
                data=eval_df,
                targets="target",
                model_type="regressor",
                evaluators=["default"],
            )

2024/11/21 14:28:59 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/11/21 14:28:59 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2024/11/21 14:28:59 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/11/21 14:28:59 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils di

2024/11/21 14:30:23 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/21 14:30:23 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/21 14:30:23 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/11/21 14:30:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest at: http://mlflow-service:5000/#/experiments/270645224134058005/runs/e22d5ff028c142dc83bedb8ad8f48cf3.
2024/11/21 14:30:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/270645224134058005.
2024/11/21 14:30:23 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/21 14:30:23 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
2024/11/21 14:30:23 WARNING mlflow.system_metrics.system

2024/11/21 14:30:30 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/21 14:30:30 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/21 14:30:30 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/11/21 14:30:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run LinearRegression at: http://mlflow-service:5000/#/experiments/270645224134058005/runs/949e5b1d8fcf472fb18473812eb6fdbd.
2024/11/21 14:30:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/270645224134058005.
2024/11/21 14:30:30 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/21 14:30:30 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
2024/11/21 14:30:31 WARNING mlflow.system_metrics.sy

2024/11/21 14:30:44 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/21 14:30:44 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/21 14:30:44 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/11/21 14:30:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run HistGB at: http://mlflow-service:5000/#/experiments/270645224134058005/runs/3b1fd66d0be94606b20e12412f799601.
2024/11/21 14:30:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/270645224134058005.
2024/11/21 14:30:44 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/21 14:30:44 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
2024/11/21 14:30:44 INFO mlflow.tracking._tracking_service.cli

# Работа с реджистри

In [40]:
# Запустим свой первый run в рамках созданного выше эксперимента.
with mlflow.start_run() as run:
    # Обучим модель.
    model.fit(pd.DataFrame(X_train), y_train)

    # Сделаем предсказание.
    prediction = model.predict(X_val)

    # Создадим валидационный датасет.
    eval_df = X_val.copy()
    eval_df["target"] = y_val

    # Сохраним результаты обучения с помощью MLFlow.
    signature = infer_signature(X_test, prediction)
    model_info = mlflow.sklearn.log_model(model, 'linreg', signature=signature)
    mlflow.evaluate(
        model=model_info.model_uri,
        data=eval_df,
        targets="target",
        model_type="regressor",
        evaluators=["default"],
    )

2024/11/21 14:47:20 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/11/21 14:47:20 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing d

2024/11/21 14:47:29 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/21 14:47:29 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/21 14:47:29 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/11/21 14:47:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run grandiose-elk-954 at: http://mlflow-service:5000/#/experiments/270645224134058005/runs/8f50c12444db40c09a4173effa2830ff.
2024/11/21 14:47:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/270645224134058005.
2024/11/21 14:47:29 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/21 14:47:30 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [41]:
model_info.model_uri

'runs:/8f50c12444db40c09a4173effa2830ff/linreg'

Регистрация модели и версии способ 1

In [42]:
mlflow.register_model(model_uri=model_info.model_uri, name='linreg-model')

Successfully registered model 'linreg-model'.
2024/11/21 14:51:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: linreg-model, version 1
Created version '1' of model 'linreg-model'.


<ModelVersion: aliases=[], creation_timestamp=1732200719938, current_stage='None', description='', last_updated_timestamp=1732200719938, name='linreg-model', run_id='8f50c12444db40c09a4173effa2830ff', run_link='', source='s3://kirillbucket/mlflow/270645224134058005/8f50c12444db40c09a4173effa2830ff/artifacts/linreg', status='READY', status_message='', tags={}, user_id='', version='1'>

Регистрация версии способ 2

In [43]:
client = MlflowClient()

In [44]:
client.create_model_version(name='linreg-model', source=model_info.model_uri, run_id=run.info.run_id)

2024/11/21 14:56:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: linreg-model, version 2


<ModelVersion: aliases=[], creation_timestamp=1732200986145, current_stage='None', description='', last_updated_timestamp=1732200986145, name='linreg-model', run_id='8f50c12444db40c09a4173effa2830ff', run_link='', source='runs:/8f50c12444db40c09a4173effa2830ff/linreg', status='READY', status_message='', tags={}, user_id='', version='2'>

Регистрация версии способ 3

In [45]:
# Создадим parent run.
with mlflow.start_run(run_name="parent_run", experiment_id = experiment_id, description = "parent") as parent_run:
    for model_name in models.keys():
        # Запустим child run на каждую модель.
        with mlflow.start_run(run_name=model_name, experiment_id=experiment_id, nested=True) as child_run:
            model = models[model_name]
            
            # Обучим модель.
            model.fit(pd.DataFrame(X_train), y_train)
        
            # Сделаем предсказание.
            prediction = model.predict(X_val)

            # Создадим валидационный датасет.
            eval_df = X_val.copy()
            eval_df["target"] = y_val
        
            # Сохраним результаты обучения с помощью MLFlow.
            signature = infer_signature(X_test, prediction)
            model_info = mlflow.sklearn.log_model(model, "linreg", signature=signature, 
                                                  registered_model_name=f"sk-learn-{model_name}-reg-model")
            mlflow.evaluate(
                model=model_info.model_uri,
                data=eval_df,
                targets="target",
                model_type="regressor",
                evaluators=["default"],
            )

2024/11/21 14:57:45 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/11/21 14:57:45 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2024/11/21 14:57:45 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/11/21 14:57:45 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils di

2024/11/21 14:58:42 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/21 14:58:42 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/21 14:58:42 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/11/21 14:58:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest at: http://mlflow-service:5000/#/experiments/270645224134058005/runs/70e39a0a42bb4628b9a2d2d73920ffe2.
2024/11/21 14:58:43 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/270645224134058005.
2024/11/21 14:58:43 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/21 14:58:43 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
2024/11/21 14:58:43 WARNING mlflow.system_metrics.system

2024/11/21 14:58:48 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/21 14:58:48 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/21 14:58:48 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/11/21 14:58:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run LinearRegression at: http://mlflow-service:5000/#/experiments/270645224134058005/runs/f7000d0b5f78446d95528373f4733330.
2024/11/21 14:58:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/270645224134058005.
2024/11/21 14:58:48 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/21 14:58:48 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
2024/11/21 14:58:48 WARNING mlflow.system_metrics.sy

2024/11/21 14:58:53 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/11/21 14:58:53 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/11/21 14:58:53 WARNING mlflow.models.evaluation.default_evaluator: SHAP or matplotlib package is not installed, so model explainability insights will not be logged.
2024/11/21 14:58:53 INFO mlflow.tracking._tracking_service.client: 🏃 View run HistGB at: http://mlflow-service:5000/#/experiments/270645224134058005/runs/6df62b40ae5349e6a80ef92776702bce.
2024/11/21 14:58:53 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-service:5000/#/experiments/270645224134058005.
2024/11/21 14:58:53 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/11/21 14:58:53 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
2024/11/21 14:58:53 INFO mlflow.tracking._tracking_service.cli

In [47]:
model = mlflow.sklearn.load_model(model_uri='s3://kirillbucket/mlflow/270645224134058005/6df62b40ae5349e6a80ef92776702bce/artifacts/linreg')

In [48]:
model.predict(X_test)

array([0.93816046, 3.37642535, 1.5238094 , ..., 1.56865319, 3.1925248 ,
       0.80610526])